In [1]:
import pandas as pd
import numpy as np
import os
import country_converter as coco
from typing import Optional, Tuple
import logging
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import warnings
from babel.languages import get_territory_language_info

# Suppress only FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

coco_logger = coco.logging.getLogger()
coco_logger.setLevel(logging.WARNING)

# Using geopy to get geographical coordinates
geolocator = Nominatim(user_agent="MyPersonalGeocoder/1.0 (myemail@example.com)", timeout=10)

# Dictionary mapping file names to their respective processing functions
files = {
  "NBA": ["NBA_foreign.csv", "NBA_height.csv", "NBA_weight.csv"],
  "NFL": ["NFL_all.csv", "NFL_measure.csv", "NFL_births.csv"],
  "MLB": ["MLB_foreign.csv", "MLB_measure.csv"],
  "NHL": ["NHL_foreign.csv", "NHL_measure.csv"],
  "MLS": ["MLS_foreign.csv"],
}
league_files = {}

# Global list of final column names
final_columns = ['Player', 'Player-additional', 'League', 'Season', 'Home Continent', 'Home Country', 'Home City', 'Overall Value', 'Offensive Performance', 'Defensive Performance', 'Biometrics', 'Migration Difficulty']
import pandas as pd

# Define the data types for each column
final_columns_dtype_mapping = {
    'Player': 'object',
    'Player-additional': 'object',
    'League': 'object',
    'Season': 'object', 
    'Home Continent': 'object',
    'Home Country': 'object',
    'Home City': 'object',
    'Overall Value': 'float',
    'Offensive Performance': 'float',
    'Defensive Performance': 'float',
    'Biometrics': 'float',
    'Migration Difficulty': 'float'
}


In [2]:
inaccessable_columns = {
    "Unknown": set(),
    }

def get_row_value(row, column_name: str, type_=str):
    try:
        value = row[column_name]
        if pd.isna(value):
            return np.nan
        if not isinstance(value, type_):
            try:
                return type_(value)
            except ValueError:
                return np.nan
    except KeyError:
        inaccessable_columns["Unknown"].add(column_name)
        return np.nan
    except AttributeError as e:
        print(f"AttributeError accessing row[{column_name}]: {str(e)}, with row {row}")
        return np.nan
    return value

In [3]:
# Paths for the CSV files
country_df_path = 'countries.csv'
location_df_path = 'locations.csv'
distance_df_path = 'distances.csv'

# Load or initialize the country dataframe
if os.path.exists(country_df_path):
    country_df = pd.read_csv(country_df_path)
else:
    country_df = pd.DataFrame(columns=['code', 'country_name', 'continent', 'language', 'language_tier'])

# Load or initialize the location dataframe
if os.path.exists(location_df_path):
    location_df = pd.read_csv(location_df_path)
else:
    location_df = pd.DataFrame(columns=['city', 'country', 'latitude', 'longitude'])

# Load or initialize the distance dataframe
if os.path.exists(distance_df_path):
    distance_df = pd.read_csv(distance_df_path)
else:
    distance_df = pd.DataFrame(columns=['city_coords', 'sf_distance', 'ny_distance'])


In [4]:
def get_country_name(code: str):
    global country_df
    
    match = country_df[country_df['code'] == code]
    if not match.empty and not pd.isna(match['country_name'].iloc[0]):
        return match['country_name'].iloc[0]
    
    # Call coco if no data is found or if the country name is NaN
    country_name = coco.convert(names=code, to='name_short')
    if match.empty:
        new_row = pd.DataFrame({'code': [code], 'country_name': [country_name], 'continent': [pd.NA]})
        country_df = pd.concat([country_df, new_row], ignore_index=True)
    else:
        country_df.loc[country_df['code'] == code, 'country_name'] = country_name
    return country_name

def get_continent(code: str):
    global country_df
    
    match = country_df[country_df['code'] == code]
    if not match.empty and 'continent' in match.columns and not pd.isna(match['continent'].iloc[0]):
        return match['continent'].iloc[0]
    
    # Call coco if no data is found or continent is NaN
    continent = coco.convert(names=code, to='Continent_7')
    if match.empty:
        new_row = pd.DataFrame({'code': [code], 'country_name': [pd.NA], 'continent': [continent]})
        country_df = pd.concat([country_df, new_row], ignore_index=True)
    else:
        country_df.loc[country_df['code'] == code, 'continent'] = continent
    return continent

def lookup_location(city: str, country: str) -> Tuple[float, float]:
    global location_df

    match = location_df[(location_df['city'] == city) & (location_df['country'] == country)]
    if not match.empty:
        return match[['latitude', 'longitude']].iloc[0].tolist()
    
    try:
        location = geolocator.geocode(f"{city}, {country}")
        if location:
            new_row = pd.DataFrame({'city': [city], 'country': [country], 'latitude': [location.latitude], 'longitude': [location.longitude]})
            location_df = pd.concat([location_df, new_row], ignore_index=True)
            return (location.latitude, location.longitude)
    except (GeocoderTimedOut, GeocoderUnavailable) as e:
        print(f"Geocoding error due to timeout or unavailability: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None


def get_sf_distance(city_coords: Tuple[float, float], san_francisco_coords: Tuple[float, float] = (37.7749, -122.4194)):
    global distance_df

    # Convert city coordinates to a string format for easy comparison
    city_coords_str = f"{city_coords[0]},{city_coords[1]}"

    # Check if the distance already exists in the dataframe
    match = distance_df[distance_df['city_coords'] == city_coords_str]
    if not match.empty and not pd.isna(match['sf_distance'].iloc[0]):
        return match['sf_distance'].iloc[0]
    
    # Calculate distance if no match is found
    distance = geodesic(city_coords, san_francisco_coords).kilometers
    if match.empty:
        # Add new row if no entry exists for these city coordinates
        new_row = pd.DataFrame({'city_coords': [city_coords_str], 'sf_distance': [distance], 'ny_distance': [pd.NA]})
        distance_df = pd.concat([distance_df, new_row], ignore_index=True)
    else:
        # Update existing row with SF distance if it was NaN
        distance_df.loc[distance_df['city_coords'] == city_coords_str, 'sf_distance'] = distance
    return distance

def get_ny_distance(city_coords: Tuple[float, float], new_york_coords: Tuple[float, float] = (40.7128, -74.0060)):
    global distance_df

    city_coords_str = f"{city_coords[0]},{city_coords[1]}"

    match = distance_df[distance_df['city_coords'] == city_coords_str]
    if not match.empty and not pd.isna(match['ny_distance'].iloc[0]):
        return match['ny_distance'].iloc[0]
    
    # Calculate distance if no match is found
    distance = geodesic(city_coords, new_york_coords).kilometers
    if match.empty:
        # Add new row if no entry exists for these city coordinates
        new_row = pd.DataFrame({'city_coords': [city_coords_str], 'sf_distance': [pd.NA], 'ny_distance': [distance]})
        distance_df = pd.concat([distance_df, new_row], ignore_index=True)
    else:
        # Update existing row with NY distance if it was NaN
        distance_df.loc[distance_df['city_coords'] == city_coords_str, 'ny_distance'] = distance
    return distance

def get_distance(city, country):
    city_coords = lookup_location(city, country)
    if city_coords:
        distance_to_sf = get_sf_distance(city_coords)
        distance_to_ny = get_ny_distance(city_coords)
        distance = min(distance_to_sf, distance_to_ny)
    else:
        distance = 0

    return distance

In [5]:
# language_tiers = {
#     "English": 0,
#     "Afrikaans": 1, "Danish": 1, "Dutch": 1, "French": 1, "Italian": 1, 
#     "Norwegian": 1, "Portuguese": 1, "Romanian": 1, "Spanish": 1, "Swedish": 1,
#     "German": 2,
#     "Indonesian": 3, "Malaysian": 3, "Swahili": 3,
#     "Albanian": 4, "Amharic": 4, "Armenian": 4, "Azerbaijani": 4, "Bengali": 4, 
#     "Bosnian": 4, "Bulgarian": 4, "Burmese": 4, "Croatian": 4, "Czech": 4,
#     "Estonian": 4, "Finnish": 4, "Georgian": 4, "Greek": 4, "Hebrew": 4, 
#     "Hindi": 4, "Hungarian": 4, "Icelandic": 4, "Khmer": 4, "Lao": 4, 
#     "Latvian": 4, "Lithuanian": 4, "Macedonian": 4, "Mongolian": 4, 
#     "Nepali": 4, "Pashto": 4, "Persian": 4, "Polish": 4, "Russian": 4, 
#     "Serbian": 4, "Sinhala": 4, "Slovak": 4, "Slovenian": 4, "Tagalog": 4, 
#     "Thai": 4, "Turkish": 4, "Ukrainian": 4, "Urdu": 4, "Uzbek": 4, 
#     "Vietnamese": 4, "Xhosa": 4, "Zulu": 4,
#     "Arabic": 5, "Cantonese": 5, "Mandarin": 5, "Japanese": 5, "Korean": 5
# }

language_tiers = {
    "en": 0,  # English
    "af": 1,  # Afrikaans
    "da": 1,  # Danish
    "nl": 1,  # Dutch
    "fr": 1,  # French
    "it": 1,  # Italian
    "no": 1,  # Norwegian
    "pt": 1,  # Portuguese
    "ro": 1,  # Romanian
    "es": 1,  # Spanish
    "sv": 1,  # Swedish
    "de": 2,  # German
    "id": 3,  # Indonesian
    "ms": 3,  # Malaysian
    "sw": 3,  # Swahili
    "sq": 4,  # Albanian
    "am": 4,  # Amharic
    "hy": 4,  # Armenian
    "az": 4,  # Azerbaijani
    "bn": 4,  # Bengali
    "bs": 4,  # Bosnian
    "bg": 4,  # Bulgarian
    "my": 4,  # Burmese
    "hr": 4,  # Croatian
    "cs": 4,  # Czech
    "et": 4,  # Estonian
    "fi": 4,  # Finnish
    "ka": 4,  # Georgian
    "el": 4,  # Greek
    "he": 4,  # Hebrew
    "hi": 4,  # Hindi
    "hu": 4,  # Hungarian
    "is": 4,  # Icelandic
    "km": 4,  # Khmer
    "lo": 4,  # Lao
    "lv": 4,  # Latvian
    "lt": 4,  # Lithuanian
    "mk": 4,  # Macedonian
    "mn": 4,  # Mongolian
    "ne": 4,  # Nepali
    "ps": 4,  # Pashto
    "fa": 4,  # Persian
    "pl": 4,  # Polish
    "ru": 4,  # Russian
    "sr": 4,  # Serbian
    "si": 4,  # Sinhala
    "sk": 4,  # Slovak
    "sl": 4,  # Slovenian
    "tl": 4,  # Tagalog (Filipino)
    "th": 4,  # Thai
    "tr": 4,  # Turkish
    "uk": 4,  # Ukrainian
    "ur": 4,  # Urdu
    "uz": 4,  # Uzbek
    "vi": 4,  # Vietnamese
    "xh": 4,  # Xhosa
    "zu": 4,  # Zulu
    "ar": 5,  # Arabic
    "yue": 5, # Cantonese
    "zh": 5,  # Mandarin
    "ja": 5,  # Japanese
    "ko": 5   # Korean
}

def get_lowest_tier_language(code: str):
    global country_df
    
    # Convert the provided code to ISO alpha-2 for consistency in DataFrame
    alpha_2_code = coco.convert(names=code, to='ISO2')
    
    # First, check if the data already exists in the DataFrame
    match = country_df[country_df['code'] == code]
    if not match.empty and not pd.isna(match['language'].iloc[0]) and not pd.isna(match['language_tier'].iloc[0]):
        return match['language'].iloc[0], match['language_tier'].iloc[0]

    # Fetch all languages spoken in the territory along with their details
    languages_info = get_territory_language_info(alpha_2_code)
    
    # Filter to include only official languages with significant population percentage
    official_languages = {
        lang: info for lang, info in languages_info.items()
        if info.get('official_status') and info.get('population_percent', 0) >= 30
    }

    if not official_languages:
        return None, None

    # Assuming language_tiers dictionary maps language codes to their respective tiers
    default_tier = 5
    lowest_tier_language = None
    lowest_tier = default_tier

    # Determine the language with the lowest tier
    for language, _ in official_languages.items():
        language_tier = language_tiers.get(language, default_tier)
        if language_tier <= lowest_tier:
            lowest_tier = language_tier
            lowest_tier_language = language

    # Update the DataFrame with new data if not found
    if match.empty:
        new_row = pd.DataFrame({
            'code': [code], 
            'language': [lowest_tier_language], 
            'language_tier': [lowest_tier]
        })
        country_df = pd.concat([country_df, new_row], ignore_index=True)
    else:
        country_df.loc[country_df['code'] == code, 'language'] = lowest_tier_language
        country_df.loc[country_df['code'] == code, 'language_tier'] = lowest_tier

    return lowest_tier_language, lowest_tier



In [6]:
def save_geo_dataframes():
    country_df.to_csv(country_df_path, index=False)
    location_df.to_csv(location_df_path, index=False)
    distance_df.to_csv(distance_df_path, index=False)


In [7]:
# Generic

def extract_player_additional_generic(row):
    player = get_row_value(row, "Player", type_=str)
    player_add =  get_row_value(row, "Player-additional", type_=str)
    season = get_row_value(row, "Season", type_=str)
    return (player, player_add, season)


# Define a dictionary for U.S. and Canadian state codes to their country ISO codes
state_to_country = {
    # U.S. state codes (abbreviated for brevity)
    "AL": "USA", "AK": "USA", "AZ": "USA", "AR": "USA", "CA": "USA",
    "CO": "USA", "CT": "USA", "DE": "USA", "FL": "USA", "GA": "USA",
    "HI": "USA", "ID": "USA", "IL": "USA", "IN": "USA", "IA": "USA",
    "KS": "USA", "KY": "USA", "LA": "USA", "ME": "USA", "MD": "USA",
    "MA": "USA", "MI": "USA", "MN": "USA", "MS": "USA", "MO": "USA",
    "MT": "USA", "NE": "USA", "NV": "USA", "NH": "USA", "NJ": "USA",
    "NM": "USA", "NY": "USA", "NC": "USA", "ND": "USA", "OH": "USA",
    "OK": "USA", "OR": "USA", "PA": "USA", "RI": "USA", "SC": "USA",
    "SD": "USA", "TN": "USA", "TX": "USA", "UT": "USA", "VT": "USA",
    "VA": "USA", "WA": "USA", "WV": "USA", "WI": "USA", "WY": "USA",
    # Canadian province codes
    "AB": "CAN", "BC": "CAN", "MB": "CAN", "NB": "CAN", "NL": "CAN",
    "NS": "CAN", "ON": "CAN", "PE": "CAN", "QC": "CAN", "SK": "CAN",
    "NT": "CAN", "NU": "CAN", "YT": "CAN",
    # Other codes
    "SUN": "RUS", "CRK": "CZE", "ENG": "GBR", "CSK": "SVK", "DDR": "DEU", "YUG": "SRB"
}

# Earth circumference assumptions
MAX_DISTANCE = 20037.5  # Half the Earth's circumference in km

def extract_location_generic(row) -> Tuple[Optional[str], Optional[str], Optional[str], Optional[int]]:
    """
    Extracts values for columns 'Home Continent', 'Home Country', 'Home City', "Migration Difficulty"
    """
    code = get_row_value(row, 'Birth Location', type_=str)
    if pd.isna(code) or "":
        return (np.nan, np.nan, np.nan, np.nan)

    # Split from the back and only split once
    city, code = code.rsplit(' ', 1)

    # First check against state codes, then use country_converter if needed
    code = state_to_country.get(code, code)

    # Find country names
    country = get_country_name(code)
    if country == "not_found":
        print(f"Country code {code} not found in row {row}")
        country = np.nan

    # Convert code to continent
    continent = get_continent(code)
    if continent == "not_found":
        print(f"Continent code {code} not found in row {row}")
        continent = np.nan

    language, language_tier = get_lowest_tier_language(code)
    if not language or not language_tier:
        language = np.nan
        language_tier = 0

    language_tier = language_tier / 5 # normalize to [0, 1]

    distance = get_distance(city, country)
    distance_norm = distance / MAX_DISTANCE # normalize to [0, 1]

    migration = f"{language}:{language_tier} {distance}:{distance_norm}"

    return (continent, country, city, migration)


In [8]:
# Overall Value

# Constants for maximum values used in normalization
SCORE_WS = (-3, 26) # Win Shares (Woody Sauldsberry -2.8 1960-61) - (Kareem Abdul-Jabbar 25.4 1971-72)
SCORE_AV = (-6, 26) # Approximate Value (Dan Pastorini -6 1981) - (LaDainian Tomlinson 26 2006)
SCORE_WAR = (-6, 21) # Wins Above Replacement (Jersey Bakley -5.3 1884) - (Pud Galvin 20.5 1884)
PLUS_MINUS_MLS = (-38, 48) # Plus-Minus (Mathieu Deplagne -38 2019) - (Carlos Vela 48 2019)
SCORE_PS = (-2.4, 23) # Point Shares (Ken Baumgartner -2.4 1997-98) - (Bobby Orr 22.8 1970-71)

def extract_overall_value_nba(row):
    win_shares = get_row_value(row, 'WS', float)
    all_star_appearances = get_row_value(row, 'AS', float)

    # Convert NaN to 0
    win_shares = 0 if np.isnan(win_shares) else win_shares
    all_star_appearances = 0 if np.isnan(all_star_appearances) else all_star_appearances

    # Normalize with new range
    win_shares_norm = (win_shares - SCORE_WS[0]) / (SCORE_WS[1] - SCORE_WS[0]) * 100
    score_all_star = all_star_appearances * 100

    return f"{win_shares}:{win_shares_norm} {all_star_appearances}:{score_all_star}"

def extract_overall_value_nfl(row):
    av = get_row_value(row, 'AV', float)

    # Convert NaN to 0
    av = 0 if np.isnan(av) else av

    # Normalize with new range
    av_score = (av - SCORE_AV[0]) / (SCORE_AV[1] - SCORE_AV[0]) * 100
    return f"{av}:{av_score} "

def extract_overall_value_mlb(row):
    war = get_row_value(row, 'WAR', float)

    # Convert NaN to 0
    war = 0 if np.isnan(war) else war

    # Normalize with new range
    war_score = (war - SCORE_WAR[0]) / (SCORE_WAR[1] - SCORE_WAR[0]) * 100
    return f"{war}:{war_score} "

def extract_overall_value_mls(row):
    plus_minus = get_row_value(row, '+/-', int)

    # Convert NaN to 0
    plus_minus = 0 if np.isnan(plus_minus) else plus_minus

    # Normalize with new range
    plus_minus_score = (plus_minus - PLUS_MINUS_MLS[0]) / (PLUS_MINUS_MLS[1] - PLUS_MINUS_MLS[0]) * 100
    return f"{plus_minus}:{plus_minus_score} "

def extract_overall_value_nhl(row):
    ps = get_row_value(row, 'PS', float)

    # Convert NaN to 0
    ps = 0 if np.isnan(ps) else ps

    # Normalize with new range
    ps_score = (ps - SCORE_PS[0]) / (SCORE_PS[1] - SCORE_PS[0]) * 100
    return f"{ps}:{ps_score} "


In [9]:
# Constants for maximum values used in normalization
MIN_NBA_PTS, MAX_NBA_PTS = 0, 4029 # Points (Wilt Chamberlain 4029 1961-62)
MIN_NBA_PPG, MAX_NBA_PPG = 0, 51 # Points per Game (Wilt Chamberlain 50.4 1961-62)
MIN_NBA_AST, MAX_NBA_AST = 0, 1164 # Assists (John Stockton 1164 1990-91)
MIN_NBA_APG, MAX_NBA_APG = 0, 15 # Assists per Game (John Stockton 14.5 1989-90)

MIN_NFL_PTS, MAX_NFL_PTS = 0, 186 # Points (LaDainian Tomlinson 186 2006)
MIN_NFL_PPG, MAX_NFL_PPG = 0, 14.7 # Points per Game (Paul Hornung 176 12 Games 1960)
MIN_NFL_YSCM, MAX_NFL_YSCM = -23, 2509 # Yards from Scrimmage (Chris Johnson 2509 2009)
MIN_NFL_YSCMPG, MAX_NFL_YSCMPG = 0, 157 # Yards from Scrimmage per Game (Chris Johnson 2509 16 Games 2009)
MIN_NFL_PYDS, MAX_NFL_PYDS = -4, 5477 # Passing Yards (Peyton Manning 5477 2013)
MIN_NFL_PYDSPG, MAX_NFL_PYDSPG = -4, 343 # Passing Yards per Game (Peyton Manning 5477 16 Games 2013)

MIN_MLB_HR, MAX_MLB_HR = 0, 73 # Home Runs (Barry Bonds 73 2001)
MIN_MLB_HRPG, MAX_MLB_HRPG = 0, 0.5 # Home Runs per Game (Barry Bonds 73 153 Games 2001)
MIN_MLB_RBI, MAX_MLB_RBI = 0, 160 # Runs Batted In (Hack Wilson 191 1930)
MIN_MLB_RBIPG, MAX_MLB_RBIPG = 0, 1.25 # Runs Batted In per Game (Hack Wilson 191 155 Games 1930)

MIN_MLS_GOALS, MAX_MLS_GOALS = 0, 34 # Goals (Carlos Vela 34 2019)
MIN_MLS_GOALSPG, MAX_MLS_GOALSPG = 0, 1.1 # Goals per Game (Carlos Vela 34 31 Games 2019)
MIN_MLS_AST, MAX_MLS_AST = 0, 21 # Assists (Carlos Valderrama 21 2000)
MIN_MLS_ASTPG, MAX_MLS_ASTPG = 0, 1.0 # Assists per Game (Lionel Messi 10 10 Games 2024)

MIN_NHL_POINTS, MAX_NHL_POINTS = 0, 215 # Points (Wayne Gretzky 215 1985-86)
MIN_NHL_POINTSPG, MAX_NHL_POINTSPG = 0, 2.7 # Points per Game (Wayne Gretzky 215 80 Games 1985-86)
MIN_NHL_GOALS, MAX_NHL_GOALS = 0, 92 # Goals (Wayne Gretzky 92 1981-82)
MIN_NHL_GOALSPG, MAX_NHL_GOALSPG = 0, 1.15 # Goals per Game (Wayne Gretzky 92 80 Games 1981-82)

def extract_offensive_performance_nba(row):
    g = get_row_value(row, 'G', float)
    pts = get_row_value(row, 'PTS', float)
    ast = get_row_value(row, 'AST', float)
    if pd.isna(g) or g == 0:
        return np.nan
    if pd.isna(pts):
        pts = 0
    if pd.isna(ast):
        ast = 0
    
    ppg = pts / g
    apg = ast / g
    p_score = 100 * (pts - MIN_NBA_PTS) / (MAX_NBA_PTS - MIN_NBA_PTS)
    a_score = 100 * (ast - MIN_NBA_AST) / (MAX_NBA_AST - MIN_NBA_AST)
    ppg_score = 100 * (ppg - MIN_NBA_PPG) / (MAX_NBA_PPG - MIN_NBA_PPG)
    apg_score = 100 * (apg - MIN_NBA_APG) / (MAX_NBA_APG - MIN_NBA_APG)
    return f"{pts}:{p_score} {ast}:{a_score} {ppg}:{ppg_score} {apg}:{apg_score}"

def extract_offensive_performance_nfl(row):
    g = get_row_value(row, 'G', float)
    pts = get_row_value(row, 'Pts', int) # Total Points from all means
    yscm = get_row_value(row, 'YScm', int)  # Yards from Scrimmage
    pyds = get_row_value(row, 'Yds', int)  # Passing Yards
    if pd.isna(g) or g == 0:
        return np.nan
    if pd.isna(pts):
        pts = 0
    if pd.isna(yscm):
        yscm = 0
    if pd.isna(pyds):
        pyds = 0

    ppg = pts / g
    yscmpg = yscm / g
    pydspg = pyds / g
    pts_score = 100 * (pts - MIN_NFL_PTS) / (MAX_NFL_PTS - MIN_NFL_PTS)
    yscm_score = 100 * (yscm - MIN_NFL_YSCM) / (MAX_NFL_YSCM - MIN_NFL_YSCM)
    pyds_score = 100 * (pyds - MIN_NFL_PYDS) / (MAX_NFL_PYDS - MIN_NFL_PYDS)
    ppg_score = 100 * (ppg - MIN_NFL_PPG) / (MAX_NFL_PPG - MIN_NFL_PPG)
    yscmpg_score = 100 * (yscmpg - MIN_NFL_YSCMPG) / (MAX_NFL_YSCMPG - MIN_NFL_YSCMPG)
    pydspg_score = 100 * (pydspg - MIN_NFL_PYDSPG) / (MAX_NFL_PYDSPG - MIN_NFL_PYDSPG)
    return f"{pts}:{pts_score} {yscm}:{yscm_score} {pyds}:{pyds_score} {ppg}:{ppg_score} {yscmpg}:{yscmpg_score} {pydspg}:{pydspg_score}"

def extract_offensive_performance_mlb(row):
    g = get_row_value(row, 'G', float)
    hr = get_row_value(row, 'HR', int)
    rbi = get_row_value(row, 'RBI', int)
    if pd.isna(g) or g == 0:
        return np.nan
    if pd.isna(hr):
        hr = 0
    if pd.isna(rbi):
        rbi = 0

    hrpg = hr / g
    rbipg = rbi / g
    hr_score = 100 * (hr - MIN_MLB_HR) / (MAX_MLB_HR - MIN_MLB_HR)
    rbi_score = 100 * (rbi - MIN_MLB_RBI) / (MAX_MLB_RBI - MIN_MLB_RBI)
    hrpg_score = 100 * (hrpg - MIN_MLB_HRPG) / (MAX_MLB_HRPG - MIN_MLB_HRPG)
    rbipg_score = 100 * (rbipg - MIN_MLB_RBIPG) / (MAX_MLB_RBIPG - MIN_MLB_RBIPG)
    return f"{hr}:{hr_score} {rbi}:{rbi_score} {hrpg}:{hrpg_score} {rbipg}:{rbipg_score}"


def extract_offensive_performance_mls(row):
    g = get_row_value(row, 'MP', float)
    goals = get_row_value(row, 'Gls', int)
    assists = get_row_value(row, 'Ast', int)
    if pd.isna(g) or g == 0:
        return np.nan
    if pd.isna(goals):
        goals = 0
    if pd.isna(assists):
        assists = 0

    goalspg = goals / g
    assistspg = assists / g
    goals_score = 100 * (goals - MIN_MLS_GOALS) / (MAX_MLS_GOALS - MIN_MLS_GOALS)
    assists_score = 100 * (assists - MIN_MLS_AST) / (MAX_MLS_AST - MIN_MLS_AST)
    goalspg_score = 100 * (goalspg - MIN_MLS_GOALSPG) / (MAX_MLS_GOALSPG - MIN_MLS_GOALSPG)
    assistspg_score = 100 * (assistspg - MIN_MLS_ASTPG) / (MAX_MLS_ASTPG - MIN_MLS_ASTPG)
    return f"{goals}:{goals_score} {assists}:{assists_score} {goalspg}:{goalspg_score} {assistspg}:{assistspg_score}"


def extract_offensive_performance_nhl(row):
    g = get_row_value(row, 'GP', float)
    points = get_row_value(row, 'PTS', int)
    goals = get_row_value(row, 'G', int)
    if pd.isna(g) or g == 0:
        return np.nan
    if pd.isna(points):
        points = 0
    if pd.isna(goals):
        goals = 0

    pointspg = points / g
    goalspg = goals / g
    points_score = 100 * (points - MIN_NHL_POINTS) / (MAX_NHL_POINTS - MIN_NHL_POINTS)
    goals_score = 100 * (goals - MIN_NHL_GOALS) / (MAX_NHL_GOALS - MIN_NHL_GOALS)
    pointspg_score = 100 * (pointspg - MIN_NHL_POINTSPG) / (MAX_NHL_POINTSPG - MIN_NHL_POINTSPG)
    goalspg_score = 100 * (goalspg - MIN_NHL_GOALSPG) / (MAX_NHL_GOALSPG - MIN_NHL_GOALSPG)
    return f"{points}:{points_score} {goals}:{goals_score} {pointspg}{pointspg_score} {goalspg}:{goalspg_score}"



In [10]:
MIN_NBA_BLK, MAX_NBA_BLK = 0, 456  # Blocks (Mark Eaton 456 in 1984-85)
MIN_NBA_BPG, MAX_NBA_BPG = 0, 5.6  # Blocks Per Game (Mark Eaton 5.6 in 1984-85 82 Games)
MIN_NBA_STL, MAX_NBA_STL = 0, 301  # Steals (Alvin Robertson 301 in 1985-86)
MIN_NBA_SPG, MAX_NBA_SPG = 0, 3.7  # Steals Per Game (Alvin Robertson 3.7 in 1985-86 82 Games)
MIN_NBA_DRB, MAX_NBA_DRB = 0, 1111  # Defensive Rebounds (Kareem Abdul-Jabbar 1111 in 1975-76)
MIN_NBA_DRPG, MAX_NBA_DRPG = 0, 13.7  # Defensive Rebounds Per Game (Elvin Hayes 13.7 in 1973-74 81 Games)

MIN_NFL_TKL, MAX_NFL_TKL = 0, 200  # Tackles combined (Hardy Nickerson 214 1993)
MIN_NFL_TKLPG, MAX_NFL_TKLPG = 0, 13.4  # Tackles per Game (Hardy Nickerson 13.4 1993 16 Games)
MIN_NFL_SK, MAX_NFL_SK = 0, 23  # Sacks (Al "Bubba" Baker 23 in 1978)
MIN_NFL_SKPG, MAX_NFL_SKPG = 0, 1.75  # Sacks per Game (Reggie White 21 in 12 Games 1987)
MIN_NFL_INT, MAX_NFL_INT = 0, 14  # Interceptions (Night Train Lane 14 in 1952)
MIN_NFL_INTPG, MAX_NFL_INTPG = 0, 1.2  # Interceptions per Game (Night Train Lane 14 in 12 Games 1952)
MIN_NFL_FF, MAX_NFL_FF = 0, 10  # Forced Fumbles (Robert Mathis 10 2013)
MIN_NFL_FFPG, MAX_NFL_FFPG = 0, 0.63  # Forced Fumbles per Game (Robert Mathis 10 in 16 Games 2013)
MIN_NFL_PD, MAX_NFL_PD = 0, 31  # Passes Defended (Darrelle Revis 31 2009)
MIN_NFL_PDPG, MAX_NFL_PDPG = 0, 1.94  # Passes Defended per Game (Darrelle Revis 31 in 16 Games 2009)

MIN_MLB_ERA, MAX_MLB_ERA = 189, 0  # Earned Run Average 
MIN_MLB_SO, MAX_MLB_SO = 0, 513  # Strikeouts (Matt Kilroy 513 1886)
MIN_MLB_SOPG, MAX_MLB_SOPG = 0, 7.6  # Strikeouts per Game (Matt Kilroy 513 in 68 Games 1886)
MIN_MLB_SHO, MAX_MLB_SHO = 0, 16  # Shutouts (Grover Alexander 16 1916)

MIN_MLS_TKLWN, MAX_MLS_TKLWN = 0, 99  # Tackles won (Rasmus Schüller 99 2018)
MIN_MLS_TKLWNPG, MAX_MLS_TKLWNPG = 0, 3.6  # Tackles won per game (Jimmy Medranda 3.6 2016)
MIN_MLS_INT, MAX_MLS_INT = 0, 120  # Interceptions (Laurent Ciman 120 2016)
MIN_MLS_INTPG, MAX_MLS_INTPG = 0, 4.9  # Interceptions per game (Víctor Cabrera 4.9 2016 over 23 Games)
MIN_MLS_SVS, MAX_MLS_SVS = 0, 177  # Saves for goalkeepers (Scott Garlick 177 2000)
MIN_MLS_SVSPG, MAX_MLS_SVSPG = 0, 5  # Saves per game for goalkeepers (Jay Nolly 7.0 2005 over 5 Games)

MIN_NHL_HITS, MAX_NHL_HITS = 0, 386  # Hits (Jeremy Lauzon 386 2023)
MIN_NHL_HITSPG, MAX_NHL_HITSPG = 0, 4.89  # Hits per Game (Jeremy Lauzon 386 2023 in 79 Games)
MIN_NHL_BLOCK, MAX_NHL_BLOCK = 0, 283  # Blocked Shots (Kris Russell 283 2014)
MIN_NHL_BLOCKPG, MAX_NHL_BLOCKPG = 0, 3.6  # Blocked Shots (Kris Russell 283 2014 in 79 Games)

def extract_defensive_performance_nba(row):
    g = get_row_value(row, 'G', float)
    blk = get_row_value(row, 'BLK', float)
    stl = get_row_value(row, 'STL', float)
    drb = get_row_value(row, 'DRB', float)
    if pd.isna(g) or g == 0:
        return np.nan
    if pd.isna(blk):
        blk = 0
    if pd.isna(stl):
        stl = 0
    if pd.isna(drb):
        drb = 0
    
    bpg = blk / g
    spg = stl / g
    drpg = drb / g
    blk_score = 100 * (blk - MIN_NBA_BLK) / (MAX_NBA_BLK - MIN_NBA_BLK)
    stl_score = 100 * (stl - MIN_NBA_STL) / (MAX_NBA_STL - MIN_NBA_STL)
    drb_score = 100 * (drb - MIN_NBA_DRB) / (MAX_NBA_DRB - MIN_NBA_DRB)
    bpg_score = 100 * (bpg - MIN_NBA_BPG) / (MAX_NBA_BPG - MIN_NBA_BPG)
    spg_score = 100 * (spg - MIN_NBA_SPG) / (MAX_NBA_SPG - MIN_NBA_SPG)
    drpg_score = 100 * (drpg - MIN_NBA_DRPG) / (MAX_NBA_DRPG - MIN_NBA_DRPG)
    return f"{blk}:{blk_score} {stl}:{stl_score} {drb}:{drb_score} {bpg}:{bpg_score} {spg}:{spg_score} {drpg}:{drpg_score}"

def extract_defensive_performance_nfl(row):
    g = get_row_value(row, 'G', float)
    tkl = get_row_value(row, 'Comb', int) # Combined tackles
    sk = get_row_value(row, 'Sk', float)
    intc = get_row_value(row, 'Int', int)
    ff = get_row_value(row, 'FF', int)
    passd = get_row_value(row, 'PD', int)
    if pd.isna(g) or g == 0:
        return np.nan
    if pd.isna(tkl):
        tkl = 0
    if pd.isna(sk):
        sk = 0
    if pd.isna(intc):
        intc = 0
    if pd.isna(ff):
        ff = 0
    if pd.isna(passd):
        passd = 0
    
    tklpg = tkl / g
    skpg = sk / g
    intpg = intc / g
    ffpg = ff / g
    passdpg = passd / g
    tkl_score = 100 * (tkl - MIN_NFL_TKL) / (MAX_NFL_TKL - MIN_NFL_TKL)
    sk_score = 100 * (sk - MIN_NFL_SK) / (MAX_NFL_SK - MIN_NFL_SK)
    int_score = 100 * (intc - MIN_NFL_INT) / (MAX_NFL_INT - MIN_NFL_INT)
    ff_score = 100 * (ff - MIN_NFL_FF) / (MAX_NFL_FF - MIN_NFL_FF)
    passd_score = 100 * (passd - MIN_NFL_PD) / (MAX_NFL_PD - MIN_NFL_PD)
    tklpg_score = 100 * (tklpg - MIN_NFL_TKLPG) / (MAX_NFL_TKLPG - MIN_NFL_TKLPG)
    skpg_score = 100 * (skpg - MIN_NFL_SKPG) / (MAX_NFL_SKPG - MIN_NFL_SKPG)
    intpg_score = 100 * (intpg - MIN_NFL_INTPG) / (MAX_NFL_INTPG - MIN_NFL_INTPG)
    ffpg_score = 100 * (ffpg - MIN_NFL_FFPG) / (MAX_NFL_FFPG - MIN_NFL_FFPG)
    passdpg_score = 100 * (passdpg - MIN_NFL_PDPG) / (MAX_NFL_PDPG - MIN_NFL_PDPG)
    return f"{tkl}:{tkl_score} {sk}:{sk_score} {intc}:{int_score} {ff}:{ff_score} {passd}:{passd_score} {tklpg}:{tklpg_score} {skpg}:{skpg_score} {intpg}:{intpg_score} {ffpg}:{ffpg_score} {passdpg}:{passdpg_score}"

def extract_defensive_performance_mlb(row):
    g = get_row_value(row, 'G', float)
    era = get_row_value(row, 'ERA', float)
    so = get_row_value(row, 'SO', int)
    sho = get_row_value(row, 'SHO', int)
    if pd.isna(g) or g == 0:
        return np.nan
    if pd.isna(era):
        era = 0
    if pd.isna(so):
        so = 0
    if pd.isna(sho):
        sho = 0

    sopg = so / g
    era_score = 100 * (MIN_MLB_ERA - era) / (MIN_MLB_ERA - MAX_MLB_ERA)  # Reverse since lower ERA is better
    so_score = 100 * (so - MIN_MLB_SO) / (MAX_MLB_SO - MIN_MLB_SO)
    sopg_score = 100 * (sopg - MIN_MLB_SOPG) / (MAX_MLB_SOPG - MIN_MLB_SOPG)
    sho_score = 100 * (sho - MIN_MLB_SHO) / (MAX_MLB_SHO - MIN_MLB_SHO)
    return f"{era}:{era_score} {so}:{so_score} {sopg}:{sopg_score} {sho}:{sho_score}"

def extract_defensive_performance_mls(row):
    g = get_row_value(row, 'MP', float)
    tklwn = get_row_value(row, 'TklW', int)
    intl = get_row_value(row, 'Int', int)
    # svs = get_row_value(row, 'Svs', int)  # Assuming 'Svs' is used for goalkeeper saves
    if pd.isna(g) or g == 0:
        return np.nan
    if pd.isna(tklwn):
        tklwn = 0
    if pd.isna(intl):
        intl = 0
    # if pd.isna(svs):
        # svs = 0

    tklwnpg = tklwn / g
    intpg = intl / g
    # svspg = svs / g
    tklwn_score = 100 * (tklwn - MIN_MLS_TKLWN) / (MAX_MLS_TKLWN - MIN_MLS_TKLWN)
    intl_score = 100 * (intl - MIN_MLS_INT) / (MAX_MLS_INT - MIN_MLS_INT)
    # svs_score = 100 * (svs - MIN_MLS_SVS) / (MAX_MLS_SVS - MIN_MLS_SVS)
    tklwnpg_score = 100 * (tklwnpg - MIN_MLS_TKLWNPG) / (MAX_MLS_TKLWNPG - MIN_MLS_TKLWNPG)
    intpg_score = 100 * (intpg - MIN_MLS_INTPG) / (MAX_MLS_INTPG - MIN_MLS_INTPG)
    # svspg_score = 100 * (svspg - MIN_MLS_SVSPG) / (MAX_MLS_SVSPG - MIN_MLS_SVSPG)
    return f"{tklwn}:{tklwn_score} {intl}:{intl_score} {tklwnpg}:{tklwnpg_score} {intpg}:{intpg_score}"

def extract_defensive_performance_nhl(row):
    g = get_row_value(row, 'GP', float)
    hits = get_row_value(row, 'HIT', int)
    blocks = get_row_value(row, 'BL', int)
    if pd.isna(g) or g == 0:
        return np.nan
    if pd.isna(hits):
        hits = 0
    if pd.isna(blocks):
        blocks = 0

    hitspg = hits / g
    blockspg = blocks / g
    hits_score = 100 * (hits - MIN_NHL_HITS) / (MAX_NHL_HITS - MIN_NHL_HITS)
    blocks_score = 100 * (blocks - MIN_NHL_BLOCK) / (MAX_NHL_BLOCK - MIN_NHL_BLOCK)
    hitspg_score = 100 * (hitspg - MIN_NHL_HITSPG) / (MAX_NHL_HITSPG - MIN_NHL_HITSPG)
    blockspg_score = 100 * (blockspg - MIN_NHL_BLOCKPG) / (MAX_NHL_BLOCKPG - MIN_NHL_BLOCKPG)
    return f"{hits}:{hits_score} {blocks}:{blocks_score} {hitspg}:{hitspg_score} {blockspg}:{blockspg_score}"



In [11]:
def imp_height_to_metric_height(height: str) -> float:
  conv = [30.48, 2.54]
  ht = [float(x) for x in height.split("-")]
  return round(np.dot(ht, conv), 1)

In [12]:
# Constants for normalization
MIN_NBA_HEIGHT, MAX_NBA_HEIGHT = imp_height_to_metric_height("5-3"), imp_height_to_metric_height("7-7") # Height in cm (Muggsy Bogues, Manute Bol)
MIN_NBA_WEIGHT, MAX_NBA_WEIGHT = 133, 360   # Weight in lbs (Spud Webb, Sim Bhullar)

MIN_NFL_HEIGHT, MAX_NFL_HEIGHT = imp_height_to_metric_height("5-1"), imp_height_to_metric_height("7-0") # Height in cm (Jack Shapiro, Richard Sligh)
MIN_NFL_WEIGHT, MAX_NFL_WEIGHT = 119, 380   # Weight in lbs (Jack Shapiro - Daniel Faalele)

MIN_MLB_HEIGHT, MAX_MLB_HEIGHT = imp_height_to_metric_height("4-5"), imp_height_to_metric_height("6-11") # Height in cm (Charlie Hughes - Sean Hjelle)
MIN_MLB_WEIGHT, MAX_MLB_WEIGHT = 120, 320   # Weight in lbs (Candy Cummings - Walter Young)

MIN_MLS_HEIGHT, MAX_MLS_HEIGHT = imp_height_to_metric_height("5-1"), imp_height_to_metric_height("6-7") # Height in cm (Aaron Herrera - Adam Bedell)
MIN_MLS_WEIGHT, MAX_MLS_WEIGHT = 115, 236   # Weight in lbs (Luis Arriaga - Chris Seitz)

MIN_NHL_HEIGHT, MAX_NHL_HEIGHT = imp_height_to_metric_height("5-4"), imp_height_to_metric_height("6-9") # Height in cm (Nathan Gerbe - Zdeno Chara)
MIN_NHL_WEIGHT, MAX_NHL_WEIGHT = 133, 265   # Weight in lbs (Ken Doraty - Derek Boogaard)

def extract_measurables_nhl(row):
    height = get_row_value(row, 'Ht.', str)
    weight = get_row_value(row, 'Wt.', float)  
    
    if pd.isna(height) or pd.isna(weight):
        return np.nan
    
    height = imp_height_to_metric_height(height)
    
    # Normalize height and weight
    height_score = 100 * (height - MIN_NHL_HEIGHT) / (MAX_NHL_HEIGHT - MIN_NHL_HEIGHT)
    weight_score = 100 * (weight - MIN_NHL_WEIGHT) / (MAX_NHL_WEIGHT - MIN_NHL_WEIGHT)
    
    return f"{height}:{height_score} {weight}:{weight_score}"


def extract_measurables_nba(row):
    height = get_row_value(row, 'Ht.', str)  
    weight = get_row_value(row, 'Wt.', float)  

    if pd.isna(height) or pd.isna(weight):
        return np.nan
    
    height = imp_height_to_metric_height(height)
    
    # Normalize height and weight
    height_score = 100 * (height - MIN_NBA_HEIGHT) / (MAX_NBA_HEIGHT - MIN_NBA_HEIGHT)
    weight_score = 100 * (weight - MIN_NBA_WEIGHT) / (MAX_NBA_WEIGHT - MIN_NBA_WEIGHT)
    
    return f"{height}:{height_score} {weight}:{weight_score}"

def extract_measurables_nfl(row):
    height = get_row_value(row, 'Ht.', str)  
    weight = get_row_value(row, 'Wt.', float)  
    
    if pd.isna(height) or pd.isna(weight):
        return np.nan
    
    height = imp_height_to_metric_height(height)
    
    # Normalize height and weight
    height_score = 100 * (height - MIN_NFL_HEIGHT) / (MAX_NFL_HEIGHT - MIN_NFL_HEIGHT)
    weight_score = 100 * (weight - MIN_NFL_WEIGHT) / (MAX_NFL_WEIGHT - MIN_NFL_WEIGHT)
    
    return f"{height}:{height_score} {weight}:{weight_score}"

def extract_measurables_mlb(row):
    height = get_row_value(row, 'Ht.', str)  
    weight = get_row_value(row, 'Wt.', float)  
    
    if pd.isna(height) or pd.isna(weight):
        return np.nan
    
    height = imp_height_to_metric_height(height)
    
    # Normalize height and weight
    height_score = 100 * (height - MIN_MLB_HEIGHT) / (MAX_MLB_HEIGHT - MIN_MLB_HEIGHT)
    weight_score = 100 * (weight - MIN_MLB_WEIGHT) / (MAX_MLB_WEIGHT - MIN_MLB_WEIGHT)
    
    return f"{height}:{height_score} {weight}:{weight_score}"

def extract_measurables_mls(row):
    height = get_row_value(row, 'Ht.', str)  
    weight = get_row_value(row, 'Wt.', float)  
    
    if pd.isna(height) or pd.isna(weight):
        return np.nan
    
    height = imp_height_to_metric_height(height)
    
    # Normalize height and weight
    height_score = 100 * (height - MIN_MLS_HEIGHT) / (MAX_MLS_HEIGHT - MIN_MLS_HEIGHT)
    weight_score = 100 * (weight - MIN_MLS_WEIGHT) / (MAX_MLS_WEIGHT - MIN_MLS_WEIGHT)
    
    return f"{height}:{height_score} {weight}:{weight_score}"


In [13]:
# NBA
league = "NBA"

league_files[league] = {
    ("Player", "Player-additional", "Season"): extract_player_additional_generic,
    'League': lambda row: league,
    ('Home Continent', 'Home Country', 'Home City', "Migration Difficulty"): extract_location_generic,
    "Overall Value": extract_overall_value_nba,
    "Offensive Performance": extract_offensive_performance_nba,
    "Defensive Performance": extract_defensive_performance_nba,
    "Biometrics": extract_measurables_nba,
}


In [14]:
# NFL
league = "NFL"

league_files[league] = {
    ("Player", "Player-additional", "Season"): extract_player_additional_generic,
    'League': lambda row: league,
    ('Home Continent', 'Home Country', 'Home City', "Migration Difficulty"): extract_location_generic,
    "Overall Value": extract_overall_value_nfl,
    "Offensive Performance": extract_offensive_performance_nfl,
    "Defensive Performance": extract_defensive_performance_nfl,
    "Biometrics": extract_measurables_nfl,
}


In [15]:
# MLB
league = "MLB"

league_files[league] = {    
    ("Player", "Player-additional", "Season"): extract_player_additional_generic,
    'League': lambda row: league,    
    ('Home Continent', 'Home Country', 'Home City', "Migration Difficulty"): extract_location_generic,
    "Overall Value": extract_overall_value_mlb,
    "Offensive Performance": extract_offensive_performance_mlb,
    "Defensive Performance": extract_defensive_performance_mlb,
    "Biometrics": extract_measurables_mlb,
}

In [16]:
# NHL
league = "NHL"

league_files[league] = {
    ("Player", "Player-additional", "Season"): extract_player_additional_generic,
    'League': lambda row: league,
    ('Home Continent', 'Home Country', 'Home City', "Migration Difficulty"): extract_location_generic,
    "Overall Value": extract_overall_value_nhl,
    "Offensive Performance": extract_offensive_performance_nhl,
    "Defensive Performance": extract_defensive_performance_nhl,
    "Biometrics": extract_measurables_nhl,
}


In [17]:
# MLS
league = "MLS"

league_files[league] = {
    ("Player", "Player-additional", "Season"): extract_player_additional_generic,
    'League': lambda row: league,
    ('Home Continent', 'Home Country', 'Home City', "Migration Difficulty"): extract_location_generic,
    "Overall Value": extract_overall_value_mls,
    "Offensive Performance": extract_offensive_performance_mls,
    "Defensive Performance": extract_defensive_performance_mls,
    "Biometrics": extract_measurables_mls,
}


In [18]:
# Define a list of column names to update
columns_to_update = None
# columns_to_update = ['Player']
# columns_to_update = [("Player", "Player-additional", "Season")]
# columns_to_update = [('Home Continent', 'Home Country', 'Home City', 'Migration Difficulty')]
# columns_to_update = ['Overall Value']
# columns_to_update = ['Offensive Performance', 'Defensive Performance']
# columns_to_update = ['Overall Value', 'Offensive Performance', 'Defensive Performance', 'Biometrics']
# columns_to_update = ['Biometrics']

In [19]:
from ordered_set import OrderedSet

def adaptive_merge(dataframes):
    
  merged_df = pd.DataFrame(columns=['Player', 'Player-additional', 'Season'])
  broadcast_columns = ['Player', 'Birth Location', 'Ht.', 'Wt.', 'BMI', 'Pos', 'Country', 'State']
  for df in dataframes:
    if 'Season' in df.columns:
        merge_keys = ['Player-additional', 'Season']
    else:
        merge_keys = ['Player-additional']
        # Treat all columns as broadcast columns if the data does not contain a 'Season' column
        # broadcast_columns.extend([col for col in df.columns if col not in ['Player-additional', 'Season']])

    # Performing the merge with suffixes for overlapping column names
    merged_df = pd.merge(merged_df, df, on=merge_keys, how='outer', suffixes=('_x', '_y'))

    final_columns = OrderedSet()
    for column in merged_df.columns:
      if "_" in column:
        col, ind = column.rsplit("_", 1)
      else:
        col, ind = column, ""
        
      final_columns.add(col)
      
      # Filter out unique columns
      if ind != "x":
        continue

      # Merge all duplicate columns: '_x' columns with their '_y' columns
      merged_df[col] = merged_df[col + "_x"].combine_first(merged_df[col + "_y"])

    # Remove duplicate columns from final df
    merged_df = merged_df[list(final_columns)]

  # Fill in missing values by duplicating values for all seasons
  for broadcast_col in broadcast_columns:
    if broadcast_col in merged_df.columns:
      merged_df[broadcast_col] = merged_df.groupby('Player-additional')[broadcast_col].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

  merged_df.drop_duplicates(inplace=True)
  # merged_df.reset_index(drop=True, inplace=True)

  return merged_df



In [20]:
# Ensure the output directory exists
output_directory = "./formatted"
os.makedirs(output_directory, exist_ok=True)

# Initialize a list to store paths to the formatted CSV files
formatted_csv_paths = []

for league, filenames in files.items():
    print(f"Processing league: {league}")
    # Load the existing formatted CSV if it exists
    output_path = os.path.join(output_directory, f"{league}_formatted.csv")
    # Create df_final, which holds all processed final_columns
    if os.path.exists(output_path):
        df_final = pd.read_csv(output_path)
    else:
        # Initialize empty DataFrame with final columns if no file exists
        df_final = pd.DataFrame(columns=final_columns)

    dataframes = []
    for filename in filenames:
        print(f"\tReading file: {filename}")
        df = pd.read_csv(filename, low_memory=False)
        dataframes.append(df)
    
    if len(dataframes) > 1:
        df_merged = adaptive_merge(dataframes) # Contains all columns of all dataframes for one league
    else:
        df_merged = dataframes[0]

    # Remove all non-US for NFL
    if league == "NFL":
        df_merged = df_merged[df_merged['Country'] != 'USA']

    # Process the merged DataFrame with the functions defined for the league
    df_temp = pd.DataFrame() # Only contains processed columns, e.g., Offensive Performance, not all final_columns
    for column_names, function in league_files[league].items():
        # Skip columns that are not in the list of columns to update
        if columns_to_update and column_names not in columns_to_update:
            continue

        print(f"\t\tSynthesizing '{column_names}'...")
        column_names = list(column_names) if isinstance(column_names, tuple) else column_names
        df_temp[column_names] = df_merged.apply(function, axis=1, result_type='expand')

    # Ensure the merged DataFrame has the columns from the processed data
    df_final[df_temp.columns] = df_temp
    df_final = df_final.dropna(how='all')  # Drop rows with all NA values

    # Save inaccesible columns with league information and then clear unknown for next league
    inaccessable_columns[league], inaccessable_columns["Unknown"] = inaccessable_columns["Unknown"].copy(), set()

    # Save to a new CSV file for the entire league
    print(f"\tSaving {league} data to {output_path}...")
    save_geo_dataframes()
    df_final.to_csv(output_path, index=False)
    formatted_csv_paths.append(output_path)
    print()

# Combine all formatted CSV files into one big DataFrame and save it
combined_df = pd.concat([pd.read_csv(f) for f in formatted_csv_paths], ignore_index=True)
combined_output_path = os.path.join(output_directory, "combined_formatted.csv")
print(f"Saving all leagues combined data to {combined_output_path}...")
combined_df.to_csv(combined_output_path, index=False)

print(f"\nThe following columns were not accessible:")
for league, cols in inaccessable_columns.items():
    if cols:
        print(f"{league}:", cols)


Processing league: NBA
	Reading file: NBA_foreign.csv
	Reading file: NBA_height.csv
	Reading file: NBA_weight.csv
		Synthesizing '('Player', 'Player-additional', 'Season')'...
		Synthesizing 'League'...
		Synthesizing '('Home Continent', 'Home Country', 'Home City', 'Migration Difficulty')'...
		Synthesizing 'Overall Value'...
		Synthesizing 'Offensive Performance'...
		Synthesizing 'Defensive Performance'...
		Synthesizing 'Biometrics'...
	Saving NBA data to ./formatted/NBA_formatted.csv...

Processing league: NFL
	Reading file: NFL_all.csv
	Reading file: NFL_measure.csv
	Reading file: NFL_births.csv
		Synthesizing '('Player', 'Player-additional', 'Season')'...
		Synthesizing 'League'...
		Synthesizing '('Home Continent', 'Home Country', 'Home City', 'Migration Difficulty')'...
		Synthesizing 'Overall Value'...
		Synthesizing 'Offensive Performance'...
		Synthesizing 'Defensive Performance'...
		Synthesizing 'Biometrics'...
	Saving NFL data to ./formatted/NFL_formatted.csv...

Process

In [21]:
combined_df[combined_df.isna().any(axis=1)]

,Player,Player-additional,League,Season,Home Continent,Home Country,Home City,Overall Value,Offensive Performance,Defensive Performance,Biometrics,Migration Difficulty
64,Joe Abbey,AbbeJo20,NFL,NaN,NaN,NaN,NaN,0:18.75,0:0.0 0:0.9083728278041074 0:0.072979383324210...,0:0.0 0:0.0 0:0.0 0:0.0 0:0.0 0.0:0.0 0.0:0.0 ...,185.4:52.136752136752136 202.0:31.800766283524904,NaN
65,Fay Abbott,AbboFa20,NFL,NaN,NaN,NaN,NaN,0:18.75,0:0.0 0:0.9083728278041074 0:0.072979383324210...,0:0.0 0:0.0 0:0.0 0:0.0 0:0.0 0.0:0.0 0.0:0.0 ...,172.7:30.427350427350397 182.0:24.137931034482758,NaN
67,Hamza Abdullah,AbduHa20,NFL,2005.0,NaN,NaN,NaN,0.0:18.75,0:0.0 0:0.9083728278041074 0:0.072979383324210...,1:0.5 0.0:0.0 0:0.0 0:0.0 0:0.0 1.0:7.46268656...,188.0:56.58119658119657 216.0:37.16475095785441,NaN
68,Hamza Abdullah,AbduHa20,NFL,2006.0,NaN,NaN,NaN,1.0:21.875,0:0.0 0:0.9083728278041074 0:0.072979383324210...,12:6.0 0.0:0.0 0:0.0 1:10.0 0:0.0 1.0909090909...,188.0:56.58119658119657 216.0:37.16475095785441,NaN
69,Hamza Abdullah,AbduHa20,NFL,2007.0,NaN,NaN,NaN,3.0:28.125,0:0.0 0:0.9083728278041074 0:0.072979383324210...,48:24.0 0.0:0.0 0:0.0 2:20.0 5:16.129032258064...,188.0:56.58119658119657 216.0:37.16475095785441,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
39731,Julio Cascante,cc120b4f,MLS,2021,NaN,NaN,NaN,-16:25.581395348837212,2:5.882352941176471 1:4.761904761904762 0.0645...,19:19.19191919191919 29:24.166666666666668 0.6...,182.9:61.135371179039325 176.0:50.413223140495866,NaN
39736,Alvas Powell,2f91124d,MLS,2019,NaN,NaN,NaN,-17:24.418604651162788,0:0.0 0:0.0 0.0:0.0 0.0:0.0,13:13.131313131313131 11:9.166666666666666 1.0...,182.9:61.135371179039325 165.0:41.32231404958678,NaN
39753,Hwang In-beom,92fa5d28,MLS,2019,NaN,NaN,NaN,-18:23.25581395348837,3:8.823529411764707 3:14.285714285714286 0.088...,30:30.303030303030305 54:45.0 0.88235294117647...,175.3:44.54148471615723 154.0:32.231404958677686,NaN
39772,Luca Petrasso,fafeaf3d,MLS,2022,NaN,NaN,NaN,-19:22.093023255813954,0:0.0 2:9.523809523809524 0.0:0.0 0.0869565217...,13:13.131313131313131 18:15.0 0.56521739130434...,177.8:50.00000000000003 168.0:43.80165289256198,NaN


In [22]:
combined_df[combined_df['Player-additional'] == "mingya01"]

,Player,Player-additional,League,Season,Home Continent,Home Country,Home City,Overall Value,Offensive Performance,Defensive Performance,Biometrics,Migration Difficulty
